In [1]:
import time
import sys
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from collections import Counter
from itertools import product

In [3]:
# 读取数据
sys.path.append('/home/chaofan/powerknowledge/data')
from read_PLAID_data import read_processed_data

start_reading_time = time.time()

feature_select = [
    'i_mean', 'i_wave_factor', 'i_pp_rms', 'i_thd', 'pure_thd', 'P', 'Q',
    'P_F', 'i_hp1', 'z_hp1', 'i_hm2', 'z_hm2', 'i_hp2', 'z_hp2', 'i_hm3',
    'z_hm3', 'i_hp3', 'z_hp3', 'i_hm4', 'z_hm4', 'i_hp4', 'z_hp4', 'i_hm5',
    'z_hm5', 'i_hp5', 'z_hp5', 'i_hm6', 'z_hm6', 'i_hp6', 'z_hp6', 'i_hm7',
    'z_hm7', 'i_hp7', 'z_hp7'
]  # 选择所用特征量

selected_label = [
    'Air Conditioner', 'Blender', 'Coffee maker', 'Fan', 'Fridge', 'Hair Iron',
    'Hairdryer', 'Heater', 'Incandescent Light Bulb', 'Microwave',
    'Soldering Iron', 'Vacuum', 'Washing Machine', 'Water kettle'
]  # 选择所用电器

load_transformer = {}
num = 0
for item in selected_label:
    load_transformer[item] = num
    num += 1

each_file_len = 20
# selected_label = ['Air Conditioner']

x_train, y_train, index_train = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=each_file_len,
    feature_select=feature_select,
    Transformer=load_transformer,
    source='submetered_process2.1/training')  # 读取训练数据
y_train = y_train.reshape(-1, 1)

x_validation, y_validation, index_validation = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=each_file_len,
    feature_select=feature_select,
    Transformer=load_transformer,
    source='submetered_process2.1/validation')  # 读取验证数据
y_validation = y_validation.reshape(-1, 1)

x_trainval = np.concatenate((x_train, x_validation), axis=0)
y_trainval = np.concatenate((y_train, y_validation), axis=0)

x_test, y_test, index_test = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=each_file_len,
    feature_select=feature_select,
    Transformer=load_transformer,
    source='submetered_process2.1/testing')  # 读取测试数据

print('finished loading data, cost %.3fs' % (time.time() - start_reading_time))

finished loading data, cost 13.893s


In [ ]:
# test cluster
km=KMeans(n_clusters=10, random_state=1)  # 聚类中心为4个
x=np.log(x_train[:,5]).reshape(-1, 1) # 取对数
y_pred = km.fit_predict(x) # 进行聚类并输出每个样本的聚类编号
t=range(0,x.shape[0],1) 
plt.scatter(t,x, c=y_pred)
plt.show()
label=km.labels_.tolist()
print(Counter(label))

从上述结果可以看到，使用传统的Kmeans聚类并没有很明显的稀疏分割边界，说明所有电器的特征过于连续，或者聚类方式有待改进。

In [4]:
center_record = {}
centers = {}
feature_lens = 30  # 应该改成len函数取值
for f_index in range(feature_lens):  #
    # n_cluster = 4 * (2 * feature_lens - 2 * f_index) - 4 # 按照数圈的方式制定聚类中心
    n_cluster = 10
    km = KMeans(n_clusters=n_cluster, random_state=1)
    if f_index in [5, 6]:  #只有这两个特征（功率）需要取对数
        x = np.log(x_train[:, f_index]).reshape(-1, 1)
    x = x_train[:, f_index].reshape(-1, 1)
    y_pred = km.fit_predict(x).reshape(-1,
                                       1)  # 这里的y只是kmean里面的聚类中心编号，不是从小到大排列下来的编号
    y_train = np.concatenate((y_train, y_pred), axis=1)
    centers[f_index] = [i for item in km.cluster_centers_
                        for i in item]  # item是个数组，如果只有1维，再加个循环读取数值
    center_record[f_index] = np.argsort(
        centers[f_index]
    )  # 对kmeans得到的聚类中心从小到大进行排序，得到排序index，即center[record[0]]为中心最小值
y_label = y_train[:, 0]
y_cluster = y_train[:, 1:]
node_num_transform = {}
node_label_transform = {}
tmp = 1  # 好像公开数据集节点都是从1开始的
for feat, cluster in product(range(feature_lens),
                             range(n_cluster)):  #把所有特征的聚类值转化为顺序排列的整数
    node_num_transform['%03d,%03d' % (feat, cluster)] = str(tmp) #转为用字符串，方便后续存储
    node_label_transform[tmp] = np.array([feat, cluster
                                          ])  # 暂时用不到拆分的，直接用tmp来代表特征和聚类中心
    tmp += 1
del tmp

In [5]:
# 计算图特征并存储成数据


# 输入点列表，返回所有边
def calc_edge(Node_list):
    edge_list = []
    for node_A in Node_list:
        for node_B in Node_list:
            if node_B == node_A:
                continue
            edge_list.append(node_A + ',' + node_B)
    return edge_list


# 记录边的重复出现次数
def count_edge(edge_count_dict, edge_list):
    for edge in edge_list:
        if edge not in edge_count_dict.keys():
            edge_count_dict[edge] = 1
        else:
            edge_count_dict[edge] += 1
    return edge_count_dict  # key为边，格式为'%03d,%03d'；value为出现的次数


# 记录点的重复出现次数
def count_node(node_count_dict, node_list):  # node为字符串型
    for node in node_list:
        if node not in node_count_dict.keys():
            node_count_dict[node] = 1
        else:
            node_count_dict[node] += 1
    return node_count_dict


def save_graph_in_txt(node_counts, edge_counts, graph_label) -> None:
    file_A = open("/home/chaofan/powerknowledge/graph/PLAIDG/PLAIDG_A.txt",
                  'r+')
    file_graph_indicator = open(
        "/home/chaofan/powerknowledge/graph/PLAIDG/PLAIDG_graph_indicator.txt",
        'r+')
    file_graph_label = open(
        "/home/chaofan/powerknowledge/graph/PLAIDG/PLAIDG_graph_label.txt",
        'r+')
    file_node_attributes = open(
        "/home/chaofan/powerknowledge/graph/PLAIDG/PLAIDG_node_attributes.txt",
        'r+')
    file_node_labels = open(
        "/home/chaofan/powerknowledge/graph/PLAIDG/PLAIDG_node_labels.txt",
        'r+')
    file_edge_attributes = open(
        "/home/chaofan/powerknowledge/graph/PLAIDG/PLAIDG_edge_attributes.txt",
        'r+')
    index_convert = {}
    base_node_num = len(file_node_labels.readlines())
    base_graph_num = len(file_graph_label.readlines())
    file_graph_label.write(str(graph_label) + '\n')
    tmp = 1
    for key in node_counts.keys():  # key为节点类型，变成label；value为次数，变成attribute
        file_node_labels.write(str(key) + '\n')  #指示当前节点的标签（特征+中心所对应的编号）
        file_node_attributes.write(str(node_counts[key]) + '\n')  #指示当前节点的特征
        file_graph_indicator.write(str(base_graph_num + 1) +
                                   '\n')  #指示当前节点属于第几个graph
        index_convert[key] = str(base_node_num + tmp)  #用于描述A矩阵，重新定义边的节点编号
        tmp += 1
    for key in edge_counts.keys():  # key为节点对，变成A阵；value为次数，变成attribute
        key_str = key.split(',')
        node_a = index_convert[key_str[0]]
        node_b = index_convert[key_str[1]]
        file_A.write(node_a + ',' + node_b + '\n')
        file_edge_attributes.write(str(edge_counts[key]) + '\n')
    file_A.close()
    file_graph_indicator.close()
    file_graph_label.close()
    file_node_attributes.close()
    file_node_labels.close()
    file_edge_attributes.close()


file_count = 0
for i, yi_cluster in enumerate(y_cluster):
    # if i < 100:
    #     continue  # 跳过100个数？
    if i % each_file_len == 0:  #判断是否进入新的文件
        edge_counts = {}
        node_counts = {}
        y_label_tmp = y_label[i]
    nodes_temp = []

    for j, cluster_k in enumerate(yi_cluster):  # j为特征编号,cluster_k为所属类别
        cluster_k = int(cluster_k)
        nodes_temp.append(node_num_transform["%03d,%03d" %
                                             (j, cluster_k)])  # 读取当前特征的节点编号

    # calc edge
    edge_list = calc_edge(nodes_temp)
    edge_counts = count_edge(edge_counts, edge_list)
    # calc node
    node_counts = count_node(node_counts, nodes_temp)
    if i % (each_file_len - 1) == 0:
        save_graph_in_txt(node_counts, edge_counts, y_label_tmp)
        print('%04d/%04d' % (file_count, int(len(y_label) / each_file_len)))
        file_count += 1
        break


0000/0792


In [ ]:
# 画图用
import networkx as nx
from matplotlib.ticker import MultipleLocator

# 计算每个特征中心的位置，输入为特征编号，特征值排序值
def calc_pos(total_features_num, feature_index, sort_index):
    edge_len = round(2 * (total_features_num - feature_index))
    if sort_index < 0:
        return None, None
    elif sort_index < 0.5 * edge_len:
        return round(0.5 * edge_len - 1), round(sort_index)
    elif sort_index < 1.5 * edge_len - 1:
        return round(edge_len - 2 - sort_index), round(0.5 * edge_len - 1)
    elif sort_index < 2.5 * edge_len - 2:
        return round(-0.5 * edge_len), round(2 * edge_len - 3 - sort_index)
    elif sort_index < 3.5 * edge_len - 3:
        return round(sort_index - 3 * edge_len + 3), round(-0.5 * edge_len)
    elif sort_index < 4 * edge_len - 4:
        return round(0.5 * edge_len - 1), round(sort_index - 4 * edge_len + 4)

# 输入点列表，返回所有边
def calc_edge(Node_list):  
    edge_list = []
    for node_A in Node_list:
        for node_B in Node_list:
            if node_B == node_A:
                continue
            edge_list.append((node_A, node_B))
    return edge_list

# 记录边的重复出现次数
def count_edge(edge_count_dict, edge_list):  
    for edge in edge_list:
        if edge not in edge_count_dict.keys():
            edge_count_dict[edge] = 1
        else:
            edge_count_dict[edge] += 1
    return edge_count_dict

# 记录点的重复出现次数
def count_node(node_count_dict, node_list):
    for node in node_list:
        if node not in node_count_dict.keys():
            node_count_dict[node] = 1
        else:
            node_count_dict[node] += 1 
    return node_count_dict


each_file_len = 10
value_dict = {}
for i, yi_cluster in enumerate(y_cluster):
    if i < 100:
        continue # 跳过100个数？
    if i % each_file_len == 0:
        edge_counts = {}
        node_counts = {}
        feature_len = len(yi_cluster)
        Graph_i = nx.Graph(application=y_label[i])
    nodes_temp = []

    for j, cluster_j in enumerate(yi_cluster):   # j为特征编号
        cluster_j = int(cluster_j)
        cluster_index = np.argwhere(center_record[j] == cluster_j) # 找到当前特征聚类所处的排位
        cluster_index = cluster_index[0][0]
        x, y = calc_pos(feature_len, j, cluster_index)  # 计算在图的位置
        # pos.append([x, y])
        # Graph_i.add_node('%02d,%02d' % (x, y), value=centers[j][cluster_j])
        # nodes.append('%02d,%02d' % (x, y))
        nodes_temp.append('%02d,%02d' % (x, y))
        value_dict['%02d,%02d' % (x, y)] = centers[j][cluster_j] # 把对应位置的真实特征值记录

    # calc edge
    edge_list = calc_edge(nodes_temp)
    edge_counts = count_edge(edge_counts, edge_list)
    # calc node
    node_counts = count_node(node_counts, nodes_temp)

    # draw nodes
    if i % each_file_len == each_file_len - 1:
        edge_select = []
        node_select = []
        pos = []
        edge_threshold = 8 # 设置plot阈值
        node_threshold = 8 # 设置plot阈值

        for k, v in edge_counts.items():
            if v >= edge_threshold:
                edge_select.append(k)
        Graph_i.add_edges_from(edge_select) # 添加边

        for k, v in node_counts.items():
            if v >= node_threshold:
                node_select.append(k)
                b = k.split(',')
                pos.append([int(b[0]), int(b[1])])
                Graph_i.add_node(k, value=value_dict[k])    # 添加点

        figure1, ax1 = plt.subplots(1, 1) # 声明画布
        npos = dict(zip(node_select, pos)) # 生成点组
        nx.draw_networkx_nodes(Graph_i,
                               npos,
                               node_select,
                               node_size=30,
                               label=True)  # 画点
        nx.draw_networkx_edges(
            Graph_i,
            npos,
            edge_select,
        )   # 画边
        spacing = 0.5  # This can be your user specified spacing.
        minorLocator = MultipleLocator(spacing)
        # Set minor tick locations.
        ax1.set_xticks(range(-feature_lens, feature_lens+1, 1))
        ax1.set_yticks(range(-feature_lens, feature_lens+1, 1))
        plt.xlim(-feature_lens-0.5, feature_lens+0.5)
        plt.ylim(-feature_lens-0.5, feature_lens+0.5)
        ax1.yaxis.set_minor_locator(minorLocator)
        ax1.xaxis.set_minor_locator(minorLocator)
        # Set grid to use minor tick locations.
        ax1.grid(which='minor')
        plt.show()
        figure1.clear()
        plt.close()